<a href="https://colab.research.google.com/github/chapSKor/basicRAGs/blob/main/RAG_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install datasets transformers sentence-transformers scikit-learn torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
from transformers import T5ForConditionalGeneration, T5Tokenizer
import numpy as np

In [4]:
# Step 1: Load a text-based dataset (e.g., the 'bookcorpus' dataset)
dataset = load_dataset('ag_news', split='train')
corpus = dataset['text'][:1000]  # Use a subset for demonstration (can adjust this as needed)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [5]:
# Step 2: Load the Sentence Transformer model for retrieval
retriever_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# Encode the entire corpus using the Sentence Transformer model
corpus_embeddings = retriever_model.encode(corpus, convert_to_tensor=True)

In [7]:
# Step 3: Load the T5 model and tokenizer for text generation
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')
t5_tokenizer = T5Tokenizer.from_pretrained('t5-base')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [14]:
# Function to retrieve relevant documents from the corpus
def retrieve_passages(query, top_k=3):
    # Get the embedding of the query
    query_embedding = retriever_model.encode(query, convert_to_tensor=True)

    # Calculate cosine similarity between the query embedding and the corpus embeddings
    similarities = util.cos_sim(query_embedding, corpus_embeddings)

    # Get the top-k indices with highest similarity scores
    top_k_indices = torch.topk(similarities, k=top_k).indices.flatten()  # Flatten the tensor

    # Convert the flattened tensor indices to Python integers and retrieve the passages
    retrieved_passages = [corpus[idx.item()] for idx in top_k_indices]

    return retrieved_passages

In [15]:
# Function to generate a response using T5
def generate_response(query, retrieved_passages):
    # Combine the query and retrieved passages
    context = " ".join(retrieved_passages)
    input_text = f"question: {query} context: {context}"

    # Encode and generate using T5
    input_ids = t5_tokenizer.encode(input_text, return_tensors='pt', max_length=512, truncation=True)
    output_ids = t5_model.generate(input_ids, max_length=100, num_beams=5, early_stopping=True)
    response = t5_tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response

In [19]:
# Example Usage
query = "What is the impact of climate change on agriculture?"
retrieved_passages = retrieve_passages(query, top_k=10)
#You can adjust the top_k parameter in retrieve_passages() to retrieve more or fewer passages.
response = generate_response(query, retrieved_passages)

In [20]:
print("\nQuery:", query)
print("\nRetrieved Passages:")
for passage in retrieved_passages:
    print("-", passage)
print("\nGenerated Response:", response)


Query: What is the impact of climate change on agriculture?

Retrieved Passages:
- News: Climate Change Could Doom Alaska's Tundra In the next 100 years, Alaska will experience a massive loss of its historic tundra, as global warming allows these vast regions of cold, dry, lands to support forests and other vegetation that will dramatically alter native ecosystems. (Oregon State University press release)
- News: Warmer Weather, Human Disturbances Interact to Change Forests While a rapidly changing climate may alter the composition of northern Wisconsin's forests, disturbances such as logging also will play a critical role in how these sylvan ecosystems change over time. (University of Wisconsin-Madison press release)
- News: Duke Study Disputes Idea that Trees Can 'Relocate' Quickly in Response to Climate Change In a study with implications for how North American trees might respond to a changing climate, molecular information collected by Duke University researchers refutes a widely 